In [1]:
# run this cell if you are in colab with a single notebook opened, otherwise ignore this cell

#!git clone https://github.com/CowboyPhilip/HPML-Energy-Efficient-LLM
#%cd HPML-Energy-Efficient-LLM
!ls

sample_data


In [2]:
# 1. Install dependencies
!pip install --upgrade pip setuptools
!pip install \
    transformers \
    bitsandbytes \
    zeus-ml \
    torch \
    datasets \
    evaluate \
    scikit-learn \
    geocoder \
    requests \
    numpy \
    wandb

  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached zeus_ml-0.11.0.post1-py3-none-any.whl.metadata (8.9 kB)
  Using cached datasets-3.5.1-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu

In [ ]:
!pip install \
    flash-attn==2.0.5 \
    triton==2.0.0 \
    vllm

  Using cached flash_attn-2.0.5.tar.gz (2.3 MB)
  Preparing metadata (setup.py) ... done
  Using cached triton-2.0.0-1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.0 kB)
  Using cached vllm-0.8.5.post1-cp38-abi3-manylinux1_x86_64.whl.metadata (14 kB)
  Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.0 kB)
  Using cached lit-18.1.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached blake3-1.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached prometheus_fastapi_instrumentator-7.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached tiktoken-0.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached lm_format_enforcer-0.10.11-py3-none-any.whl.metadata (17 kB)
  Using cached llguidance-0.7.19-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using

In [16]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jy3475 (HPML-Energy-Efficient-LLM) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [17]:
!wandb status

Current Settings
{
  "_extra_http_headers": null,
  "_proxies": null,
  "api_key": null,
  "base_url": "https://api.wandb.ai",
  "entity": null,
  "git_remote": "origin",
  "ignore_globs": [],
  "organization": null,
  "project": null,
  "root_dir": null,
  "section": "default"
}


In [ ]:
# import wandb

# wandb.init(project="HPML-Energy-Efficient-LLM", name="test-connection")

# wandb.log({"test_value": "init wandb"})

# wandb.finish()

In [20]:
# global configuration for experiments
cfg = {
    "task":           "math",
    "model":          "deepseek-ai/deepseek-coder-1.3b-instruct",
    # test adaptive quant
    "modes": [
        "adaptive"
        # "fp32_vanilla",    # FP16 + vanilla Transformer
        # "fp16_vanilla",    # FP16 + vanilla Transformer
        # "int8_vanilla",    # INT8 + vanilla
        # "int4_vanilla",    # INT4 + vanilla&
    ],
    # for adaptive mode: which two modes to switch between
    "high_mode":      "fp16_vanilla",
    "low_mode":       "int8_vanilla",
    "ctx_threshold": 512,
    "latency_threshold": 0.08,

    "dataset_name":   "deepmind/math_dataset",
    "dataset_config": "algebra__linear_1d",
    "split":          "test",
    "num_examples":   20,

    "temperature": 0.5,
    "top_p":0.9,
    "verbose":        True,
    "output_file":    "results.json",
    "device_map": "cuda"

}



In [11]:
# global configuration for experiments
cfg = {
    "task":           "math",
    "model":          "deepseek-ai/deepseek-coder-1.3b-instruct",
    # test default (vanilla) kernel
    "modes": [
        "fp32_vanilla",    # FP16 + vanilla Transformer
        "fp16_vanilla",    # FP16 + vanilla Transformer
        "int8_vanilla",    # INT8 + vanilla
        "int4_vanilla",    # INT4 + vanilla&
    ],

    "dataset_name":   "deepmind/math_dataset",
    "dataset_config": "algebra__linear_1d",
    "split":          "test",
    "num_examples":   20,

    "temperature": 0.5,
    "top_p":0.9,
    "verbose":        True,
    "output_file":    "results.json",
    "device_map": "cuda"

}

In [6]:
# exp for mbpp
cfg = {
    "task":           "mbpp",
    # "model":          "deepseek-ai/deepseek-coder-1.3b-instruct",
    "model":       "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    # test default (vanilla) kernel at fp16/int8/int4, plus adaptive switching
    "modes":          [
        # "fp16_flash-v2",    # FP16 + flash attn v2 Transformer
        "fp16",
        "int8_vanilla",    # INT8 + vanilla
        "int4_vanilla",    # INT4 + vanilla& low_mode
    ],
    # for adaptive mode: which two modes to switch between
    # "high_mode":      "fp16_vanilla",
    # "low_mode":       "int8_vanilla",
    "temperature": 0.5,
    "top_p":0.9,
    "num_examples":   500,
    "quick":          True,
    "max_samples":    500,
    "batch_size":     8,
    "verbose":        True,
    "output_file":    "mbpp_results.json"
}

In [2]:
import json
import time
import numpy as np
from pathlib import Path
from tqdm import tqdm
from datasets import load_dataset  # ensure load_dataset is defined

# benchmark functions
from utils.test_generation import quick_test_generation, test_generation_MATH, test_generation_MBPP
from utils.test_mmlu    import quick_test_mmlu, test_quantized_models_on_mmlu
from utils.test_glue    import test_quantized_models_on_glue

# energy & tracking
from utils.energy_utils   import EnergyTracker, get_carbon_intensity, joules_to_co2
from utils.memory_utils   import clean_memory

# adaptive quant wrapper
from utils.adaptive_quant  import AdaptiveQuantGenerator

# plotting
from utils.plot_utils    import plot_energy_comparison, plot_component_energy

%load_ext autoreload
%autoreload 2

/opt/conda/envs/vLLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/opt/rocm/lib/libamd_smi.so: cannot open shared object file: No such file or directory
Unable to find libamd_smi.so library try installing amd-smi-lib from your package manager


In [3]:
# Monkey-patch EnergyTracker to support `with tracker:` and save_results
def _et_enter(self):
    if getattr(self, 'zeus', None):
        try:
            self.zeus.begin_window('inference')
            self.active_windows.add('inference')
        except:
            pass
    self._enter_ts = time.time()
    return self

def _et_exit(self, exc_type, exc_val, exc_tb):
    end_ts = time.time()
    inf_e = 0
    if getattr(self, 'zeus', None) and 'inference' in self.active_windows:
        try:
            m = self.zeus.end_window('inference')
            inf_e = m.total_energy
            self.active_windows.remove('inference')
        except:
            pass
    elapsed = end_ts - getattr(self, '_enter_ts', end_ts)
    comp = {k: np.sum(v) for k, v in self.comp_energy.items()}
    self.stats = {
        'total_energy': inf_e,
        'time': elapsed,
        'components': comp,
        'num_tokens': None
    }
    return False

def _save_results(self, extra_metrics):
    if not hasattr(self, 'stats'):
        self.stats = {}
    self.stats.update(extra_metrics)

EnergyTracker.__enter__    = _et_enter
EnergyTracker.__exit__     = _et_exit
EnergyTracker.save_results = _save_results

In [4]:
def run_task(cfg):
    """Dispatch benchmarks based on cfg['task'], with adaptive-quant support."""
    task = cfg["task"]
    modes = list(cfg["modes"])
    results = {}

    # unpack shared params
    high_mode  = cfg.get("high_mode")
    low_mode   = cfg.get("low_mode")
    ctx_th     = cfg.get("ctx_threshold", 1024)
    lat_th     = cfg.get("latency_threshold", 0.08)
    device_map = cfg.get("device_map", "auto")
    temp       = cfg.get("temperature", 0.5)
    top_p      = cfg.get("top_p", 0.9)
    carbon_int = get_carbon_intensity()

    # skip adaptive for pure classification
    if task in ("glue", "mmlu") and "adaptive" in modes:
        print("⚠️  Skipping adaptive for classification tasks")
        modes.remove("adaptive")

    # -------------------------
    # TEXT GENERATION
    # -------------------------
    if task == "generation":
        results["generation"] = {}

        # adaptive generation
        if "adaptive" in modes:
            print("\n=== ADAPTIVE generation ===")
            agent = AdaptiveQuantGenerator(
                cfg["model"],
                high_mode=high_mode,
                low_mode=low_mode,
                ctx_threshold=ctx_th,
                latency_threshold=lat_th,
                device_map=device_map
            )
            _, _, stats = agent.generate(
                cfg["prompt"],
                max_new_tokens=cfg["tokens"],
                temperature=temp,
                top_p=top_p
            )
            results["generation"]["adaptive"] = stats
            modes.remove("adaptive")

        # static generation modes
        for mode in modes:
            print(f"\n=== {mode.upper()} generation ===")
            stats = quick_test_generation(
                model_name=cfg["model"],
                quant_mode=mode,
                prompt=cfg["prompt"],
                max_new_tokens=cfg["tokens"],
                temperature=temp,
                top_p=top_p
            )
            results["generation"][mode] = stats

    # -------------------------
    # MATH (generation-style)
    # -------------------------
    elif task == "math":
        results["math"] = {}

        # adaptive MATH
        if "adaptive" in modes:
            print("\n=== ADAPTIVE on MATH ===")
            ds = load_dataset(
                cfg["dataset_name"],
                cfg["dataset_config"],
                split=cfg["split"]
            ).select(range(cfg["num_examples"]))

            agent = AdaptiveQuantGenerator(
                cfg["model"],
                high_mode=high_mode,
                low_mode=low_mode,
                ctx_threshold=ctx_th,
                latency_threshold=lat_th,
                device_map=device_map
            )

            examples, correct, total_tokens = [], 0, 0
            for ex in tqdm(ds, desc="Adaptive MATH"):
                prompt = f"<｜User｜>{ex['question']}<｜Assistant｜><think>"
                gen_ids, _, stats = agent.evaluate(
                    prompt,
                    agent.tokenizer,
                    max_new_tokens=cfg.get("max_new_tokens", 32),
                    temperature=temp,
                    top_p=top_p
                )
                inp_len = stats["input_length"]
                pred = agent.tokenizer.decode(
                    gen_ids[0, inp_len:], skip_special_tokens=True
                ).strip()

                is_corr = (pred == ex["answer"].strip())
                correct += int(is_corr)
                total_tokens += stats.get("num_tokens", 1)
                examples.append({
                    "question":   ex["question"],
                    "prediction": pred,
                    "is_correct": is_corr,
                    "stats":      stats
                })
                clean_memory()

            n = len(examples)
            total_e = sum(e["stats"]["total_energy"] for e in examples)
            total_t = sum(e["stats"]["time"]         for e in examples)
            results["math"]["adaptive"] = {
                "examples": examples,
                "summary": {
                    "accuracy":         100 * correct / n,
                    "avg_energy":       total_e / n,
                    "avg_time":         total_t / n,
                    "energy_per_token": total_e / total_tokens,
                    "carbon_emissions": joules_to_co2(total_e, carbon_int)
                }
            }
            plot_component_energy(results, task_type="math", quant_mode="adaptive")
            modes.remove("adaptive")

        # static MATH modes
        if modes:
            print(f"\n=== standard modes on MATH: {modes} ===")
            std = test_generation_MATH(
                model_name=cfg["model"],
                quantization_modes=modes,
                dataset_name=cfg["dataset_name"],
                dataset_config=cfg["dataset_config"],
                split=cfg["split"],
                num_examples=cfg["num_examples"],
                verbose=cfg.get("verbose", True)
            )
            results["math"].update(std)

    # -------------------------
    # MBPP
    # -------------------------
    elif task == "mbpp":
        print("\n=== MBPP task ===")
        results["mbpp"] = {}

        # adaptive MBPP
        if "adaptive" in modes:
            print("\n=== ADAPTIVE on MBPP ===")
            ds = load_dataset(
                cfg["dataset_name"],
                split=cfg.get("split", "test")
            ).select(range(cfg["num_examples"]))

            agent = AdaptiveQuantGenerator(
                cfg["model"],
                high_mode=high_mode,
                low_mode=low_mode,
                ctx_threshold=ctx_th,
                latency_threshold=lat_th,
                device_map=device_map
            )

            examples, correct, total_tokens = [], 0, 0
            for ex in tqdm(ds, desc="Adaptive MBPP"):
                prompt = ex["text"]
                gen_ids, _, stats = agent.evaluate(
                    prompt,
                    agent.tokenizer,
                    max_new_tokens=cfg.get("max_new_tokens", 128),
                    temperature=temp,
                    top_p=top_p
                )
                inp_len = stats["input_length"]
                pred = agent.tokenizer.decode(
                    gen_ids[0, inp_len:], skip_special_tokens=True
                )
                is_corr = check_mbpp(pred, ex["expected_code"])
                correct += int(is_corr)
                total_tokens += stats.get("num_tokens", 1)
                examples.append({
                    "prompt":     prompt,
                    "prediction": pred,
                    "is_correct": is_corr,
                    "stats":      stats
                })
                clean_memory()

            n = len(examples)
            total_e = sum(e["stats"]["total_energy"] for e in examples)
            total_t = sum(e["stats"]["time"]         for e in examples)
            results["mbpp"]["adaptive"] = {
                "examples": examples,
                "summary": {
                    "accuracy":         100 * correct / n,
                    "avg_energy":       total_e / n,
                    "avg_time":         total_t / n,
                    "energy_per_token": total_e / total_tokens,
                    "carbon_emissions": joules_to_co2(total_e, carbon_int)
                }
            }
            modes.remove("adaptive")

        # static MBPP modes
        std = test_generation_MBPP(
            model_name=cfg["model"],
            quantization_modes=modes,
            num_examples=cfg["num_examples"],
            verbose=cfg.get("verbose", True),
            temperature=temp,
            top_p=top_p
        )
        results["mbpp"].update(std)

    # -------------------------
    # MMLU
    # -------------------------
    elif task == "mmlu":
        print("\n=== MMLU task ===")

        if cfg.get("quick", False):
            stats = quick_test_mmlu(
                model_name=cfg["model"],
                quant_mode=modes[0],
                subjects=cfg["subjects"],
                max_samples=cfg["max_samples"]
            )
        else:
            stats = test_quantized_models_on_mmlu(
                model_name=cfg["model"],
                quantization_modes=modes,
                subjects=cfg["subjects"]
            )
        results["mmlu"] = stats

    # -------------------------
    # GLUE
    # -------------------------
    else:
        print("\n=== GLUE task ===")
        results["glue"] = test_quantized_models_on_glue(
            model_name=cfg["model"],
            tasks=cfg["glue_tasks"],
            quantization_modes=modes,
            batch_size=cfg["batch_size"]
        )

    return results

In [12]:
# results = run_task(cfg)

Location detected: Las Vegas, US (lat: 36.175, lon: -115.1372)
Using estimated carbon intensity.
Using estimate for US: 417 gCO2eq/kWh

=== standard modes on MATH: ['fp32_vanilla', 'fp16_vanilla', 'int8_vanilla', 'int4_vanilla'] ===
Location detected: Las Vegas, US (lat: 36.175, lon: -115.1372)
Using estimated carbon intensity.
Using estimate for US: 417 gCO2eq/kWh
Carbon intensity: 417 gCO2eq/kWh

=== Testing FP32_VANILLA on MATH ===
Loading FP32 model …
GPU Memory: Allocated: 0.01 GB | Reserved: 0.02 GB | Max: 4.21 GB
Model ready → quantisation: FP32, kernel: vanilla
GPU Memory: Allocated: 5.40 GB | Reserved: 5.55 GB | Max: 5.40 GB
[2025-05-07 07:41:22,008] [zeus.monitor.energy](energy.py:209) Monitoring GPU indices [0].
[2025-05-07 07:41:22,009] [zeus.monitor.energy](energy.py:210) Monitoring CPU indices []
Successfully initialized ZeusMonitor


MATH FP32_VANILLA: 100%|██████████| 20/20 [00:09<00:00,  2.00it/s]


FP32_VANILLA SUMMARY: Samples=20, Acc=0.00%,

=== Testing FP16_VANILLA on MATH ===
Loading FP16 model …
GPU Memory: Allocated: 0.01 GB | Reserved: 5.41 GB | Max: 5.43 GB
Model ready → quantisation: FP16, kernel: vanilla
GPU Memory: Allocated: 2.71 GB | Reserved: 5.41 GB | Max: 5.43 GB
[2025-05-07 07:41:34,691] [zeus.monitor.energy](energy.py:209) Monitoring GPU indices [0].
[2025-05-07 07:41:34,691] [zeus.monitor.energy](energy.py:210) Monitoring CPU indices []
Successfully initialized ZeusMonitor


MATH FP16_VANILLA: 100%|██████████| 20/20 [00:10<00:00,  1.85it/s]


FP16_VANILLA SUMMARY: Samples=20, Acc=0.00%,

=== Testing INT8_VANILLA on MATH ===
Loading INT8 model …
GPU Memory: Allocated: 0.01 GB | Reserved: 5.41 GB | Max: 5.43 GB
Model ready → quantisation: INT8, kernel: vanilla
GPU Memory: Allocated: 1.49 GB | Reserved: 5.41 GB | Max: 5.43 GB
[2025-05-07 07:41:49,209] [zeus.monitor.energy](energy.py:209) Monitoring GPU indices [0].
[2025-05-07 07:41:49,209] [zeus.monitor.energy](energy.py:210) Monitoring CPU indices []
Successfully initialized ZeusMonitor


MATH INT8_VANILLA:   0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
MATH INT8_VANILLA: 100%|██████████| 20/20 [00:10<00:00,  2.00it/s]


INT8_VANILLA SUMMARY: Samples=20, Acc=0.00%,

=== Testing INT4_VANILLA on MATH ===
Loading INT4 model …
GPU Memory: Allocated: 0.01 GB | Reserved: 5.41 GB | Max: 5.43 GB
Model ready → quantisation: INT4, kernel: vanilla
GPU Memory: Allocated: 0.91 GB | Reserved: 5.43 GB | Max: 5.43 GB
[2025-05-07 07:42:02,940] [zeus.monitor.energy](energy.py:209) Monitoring GPU indices [0].
[2025-05-07 07:42:02,941] [zeus.monitor.energy](energy.py:210) Monitoring CPU indices []
Successfully initialized ZeusMonitor


MATH INT4_VANILLA: 100%|██████████| 20/20 [00:11<00:00,  1.81it/s]


INT4_VANILLA SUMMARY: Samples=20, Acc=0.00%,


In [7]:
import wandb

wandb.init(
    project="HPML-Energy-Efficient-LLM",
    name=f"{cfg['model']}-{cfg['task']}-({'/'.join(cfg['modes'])})",
    tags=[cfg['model'].split('/')[-1], cfg['task']] + cfg['modes'],
    group=cfg['model'].split('/')[-1],
    job_type=cfg['task'],
    config=cfg
)

results = run_task(cfg)

wandb.log(results)
wandb.finish()
print(f"\nSaved results to wandb")

Location detected: São Paulo, BR (lat: -23.5475, lon: -46.6361)
Using estimated carbon intensity.
Using estimate for BR: 110 gCO2eq/kWh

=== MBPP task ===
Location detected: São Paulo, BR (lat: -23.5475, lon: -46.6361)
Using estimated carbon intensity.
Using estimate for BR: 110 gCO2eq/kWh
Carbon intensity: 110 gCO2eq/kWh

=== Testing FP16 on MBPP ===
Loading FP16 model …
GPU Memory: Allocated: 1.15 GB | Reserved: 1.79 GB | Max: 1.78 GB
Model ready → quantisation: FP16, kernel: vanilla
GPU Memory: Allocated: 4.71 GB | Reserved: 5.40 GB | Max: 4.72 GB
[2025-05-07 17:13:38,340] [zeus.device.gpu.nvidia](nvidia.py:47) pynvml is available and initialized.
[2025-05-07 17:13:38,343] [zeus.device.cpu.rapl](rapl.py:137) RAPL is not supported on this CPU.
[2025-05-07 17:13:38,344] [zeus.monitor.energy](energy.py:209) Monitoring GPU indices [0].
[2025-05-07 17:13:38,345] [zeus.monitor.energy](energy.py:210) Monitoring CPU indices []
Successfully initialized ZeusMonitor


MBPP FP16:   0%|          | 0/10 [00:00<?, ?it/s]

[2025-05-07 17:13:38,482] [zeus.utils.framework](framework.py:25) PyTorch with CUDA support is available.


MBPP FP16: 100%|██████████| 10/10 [00:07<00:00,  1.32it/s]



=== Testing INT8_VANILLA on MBPP ===
Loading INT8 model …
GPU Memory: Allocated: 1.18 GB | Reserved: 5.35 GB | Max: 5.69 GB
Model ready → quantisation: INT8, kernel: vanilla
GPU Memory: Allocated: 3.43 GB | Reserved: 5.37 GB | Max: 5.69 GB
[2025-05-07 17:14:00,194] [zeus.monitor.energy](energy.py:209) Monitoring GPU indices [0].
[2025-05-07 17:14:00,201] [zeus.monitor.energy](energy.py:210) Monitoring CPU indices []
Successfully initialized ZeusMonitor


MBPP INT8_VANILLA:   0%|          | 0/10 [00:00<?, ?it/s]/opt/conda/envs/vLLM/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
MBPP INT8_VANILLA: 100%|██████████| 10/10 [00:07<00:00,  1.30it/s]



=== Testing INT4_VANILLA on MBPP ===
Loading INT4 model …
GPU Memory: Allocated: 1.18 GB | Reserved: 5.35 GB | Max: 5.69 GB
Model ready → quantisation: INT4, kernel: vanilla
GPU Memory: Allocated: 2.80 GB | Reserved: 5.38 GB | Max: 5.69 GB
[2025-05-07 17:14:30,376] [zeus.monitor.energy](energy.py:209) Monitoring GPU indices [0].
[2025-05-07 17:14:30,377] [zeus.monitor.energy](energy.py:210) Monitoring CPU indices []
Successfully initialized ZeusMonitor


MBPP INT4_VANILLA: 100%|██████████| 10/10 [00:06<00:00,  1.48it/s]



Saved results to wandb


In [ ]:
results["mbpp"]["fp16"]["examples"][0]

{'prompt': '\n                <｜begin▁of▁sentence｜><｜User｜>output only the code, no explanation: Write a python function to remove first and last occurrence of a given character from the string.<｜Assistant｜><think>\n                ',
 'ground_truth_code': 'def remove_Occ(s,ch): \r\n    for i in range(len(s)): \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    for i in range(len(s) - 1,-1,-1):  \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    return s ',
 'generated_code': ')\n\n</think> )\n\n\n\n\n\n\n the number.\n\n no explanation\n\n  a function function that compute all occurrence last character of a character number in a string.\n\n The<think>\nOkay First',
 'test_cases': ['assert remove_Occ("hello","l") == "heo"',
  'assert remove_Occ("abcda","a") == "bcd"',
  'assert remove_Occ("PHP","P") == "H"'],
 'is_correct': False,
 'stats': {'total_energy': 30.890284558775836,
  'tokenization

In [9]:
results["math"]["int8_vanilla"]["examples"][0]

KeyError: 'int8_vanilla'

In [22]:
# print summary for each task and mode
ci = get_carbon_intensity()
for task, modes in results.items():
    print(f"\n=== {task.upper()} SUMMARY ===")
    for mode, data in modes.items():
        summary = data.get("summary", data)
        e   = summary.get("avg_energy",     summary.get("total_energy", 0.0))
        t   = summary.get("avg_time",       summary.get("total_time",   0.0))
        # acc = summary.get("accuracy",       None)
        co2 = summary.get(
            "carbon_emissions",
            joules_to_co2(summary.get("total_energy", e), ci)
        )
        line = f"{mode:>12}: E={e:.2f} J, Lat={t:.3f}s"
        # if acc is not None:
        #     line += f", Acc={acc:.2f}%"
        line += f", CO₂={co2:.4f}g"
        print(line)


Location detected: Las Vegas, US (lat: 36.175, lon: -115.1372)
Using estimated carbon intensity.
Using estimate for US: 417 gCO2eq/kWh

=== MATH SUMMARY ===
    adaptive: E=406.00 J, Lat=12.241s, CO₂=940.5700g


In [ ]:
# Plot overall energy comparison
plot_energy_comparison(results)

In [ ]:
# Plot per-component breakdown for each task and mode
for task, modes in results.items():
    for mode in modes:
        # skip modes without component stats
        stat = results[task][mode]
        comps = stat.get("summary", stat).get("components", None)
        if comps:
            plot_component_energy(results, task_type=task, quant_mode=mode)

In [ ]:
# save raw results to JSON
with open(cfg["output_file"], "w") as f:
    json.dump(results, f, indent=2)
print(f"Results saved to {cfg['output_file']}")


Results saved to results.json
